<a href="https://colab.research.google.com/github/mamendl/Affirmations/blob/master/entregable3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai -Uqq
!pip install datasets -Uqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from fastai.text.all import *
from datasets import load_dataset

In [3]:
!pip install datasets transformers

In [10]:
dataset = load_dataset("tweet_eval", "emoji")

In [11]:
train_df = dataset["train"].to_pandas()
valid_df = dataset["validation"].to_pandas()
test_df = dataset["test"].to_pandas()

In [12]:
train_df['set']=False
valid_df['set']=True
train_df = train_df.drop(columns=['label'],axis=1)
valid_df = valid_df.drop(columns=['label'],axis=1)
train_valid_df = pd.concat([train_df,valid_df])
train_valid_df = train_valid_df.rename(columns={"verse_text": "text"})

In [14]:
db_lm = DataBlock(
    blocks=TextBlock.from_df('text', is_lm=True,max_vocab=100000), # Indicamos que vamos a trabajar con un modelo de lenguaje
    get_items=ColReader('text'), # Indicamos donde estará el texto dentro del dataframe
    splitter=RandomSplitter(0.1) # Partimos el dataset en entrenamiento y validación
)

In [15]:
dls_lm = db_lm.dataloaders(train_valid_df, bs=128, seq_len=80)

In [16]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos # xxmaj versace # xxmaj thursday @ xxmaj los xxmaj angeles , xxmaj california xxbos xxmaj icecream xxmaj sandwich dipped in chocolate … .. xxmaj ridiculous @ xxmaj the xxmaj manhattan xxmaj beach xxmaj creamery xxbos ️ ️ ️ xxmaj how cute is this custom order # xxunk # handmade # custom # xxunk for # xxunk featuring … xxbos xxmaj random trip to xxmaj disneyland @ xxmaj disneyland xxbos xxmaj so is my next door neighbor 's .","# xxmaj versace # xxmaj thursday @ xxmaj los xxmaj angeles , xxmaj california xxbos xxmaj icecream xxmaj sandwich dipped in chocolate … .. xxmaj ridiculous @ xxmaj the xxmaj manhattan xxmaj beach xxmaj creamery xxbos ️ ️ ️ xxmaj how cute is this custom order # xxunk # handmade # custom # xxunk for # xxunk featuring … xxbos xxmaj random trip to xxmaj disneyland @ xxmaj disneyland xxbos xxmaj so is my next door neighbor 's . #"
1,xxmaj xxunk xxunk iloveyou ! xxmaj merry xxmaj christmas ! … xxbos xxmaj this one 's for xxunk xxunk # xxunk @ xxmaj disneyland xxbos xxmaj beautiful day in xxmaj los xxmaj angeles # streetart by @user @ xxmaj coffee xxmaj commissary xxbos # xxunk is soaking wet # rain # losangeles @ xxmaj hollywood xxmaj walk of xxmaj fame xxbos xxunk left to 2016 @ xxmaj marquee xxmaj las xxmaj vegas xxbos xxmaj having a good time in xxmaj,xxunk xxunk iloveyou ! xxmaj merry xxmaj christmas ! … xxbos xxmaj this one 's for xxunk xxunk # xxunk @ xxmaj disneyland xxbos xxmaj beautiful day in xxmaj los xxmaj angeles # streetart by @user @ xxmaj coffee xxmaj commissary xxbos # xxunk is soaking wet # rain # losangeles @ xxmaj hollywood xxmaj walk of xxmaj fame xxbos xxunk left to 2016 @ xxmaj marquee xxmaj las xxmaj vegas xxbos xxmaj having a good time in xxmaj seattle


In [17]:
dls_lm.show_batch(max_n=2)

,text,text_
0,xxbos xxmaj everyone is busy doing their christmas xxunk and here i am xxunk . # model # socal # norcal # cali … xxbos my loves @ xxmaj luther xxmaj burbank xxmaj auditorium xxbos xxmaj this pic . @ t - mobile xxmaj arena xxbos xxmaj freakin out right now xxmaj i 'm so excited xxrep 3 ! xxmaj self tanning starts today ! ! xxmaj ca nt wait to get the body … xxbos xxmaj just met the,xxmaj everyone is busy doing their christmas xxunk and here i am xxunk . # model # socal # norcal # cali … xxbos my loves @ xxmaj luther xxmaj burbank xxmaj auditorium xxbos xxmaj this pic . @ t - mobile xxmaj arena xxbos xxmaj freakin out right now xxmaj i 'm so excited xxrep 3 ! xxmaj self tanning starts today ! ! xxmaj ca nt wait to get the body … xxbos xxmaj just met the most
1,"xxmaj own it . xxmaj all or nothing . xxmaj make it absolutely fantastic . > > xxmaj repost from … xxbos @user attempting to xxunk @ xxmaj pullman , xxmaj washington xxbos xxmaj last nights magic @ xxmaj disney 's xxmaj world of xxmaj color xxbos xxmaj oh people 's xxmaj new xxmaj years xxmaj xxunk .. xxmaj should n't be a … xxbos xxmaj girls weekend in xxup sf with my two xxunk xxmaj thank you for visiting","own it . xxmaj all or nothing . xxmaj make it absolutely fantastic . > > xxmaj repost from … xxbos @user attempting to xxunk @ xxmaj pullman , xxmaj washington xxbos xxmaj last nights magic @ xxmaj disney 's xxmaj world of xxmaj color xxbos xxmaj oh people 's xxmaj new xxmaj years xxmaj xxunk .. xxmaj should n't be a … xxbos xxmaj girls weekend in xxup sf with my two xxunk xxmaj thank you for visiting /"


In [18]:
learn = language_model_learner(
    dls_lm, # El dataloader que usamos
    AWD_LSTM, # La arquitectura que es la misma usada en la práctica anterior
    drop_mult=0.3, # Aplicamos dropout para evitar el sobreajuste
    metrics=[accuracy, Perplexity()] # Como métricas usamos la accuracy y la perplexity.
).to_fp16()

In [19]:
learn.fine_tune(10,base_lr=2e-2)

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,perplexity,time


KeyboardInterrupt: 

In [ ]:
learn.save_encoder('finetuned')

In [ ]:
sentiment_clas = DataBlock(
    blocks=(TextBlock.from_df('text', vocab=dls_lm.vocab), # La entrada del modelo es texto usando el mismo
                                                           # vocabulario que en el modelo de lenguaje
            CategoryBlock), #, y la salida una clase
    get_x=ColReader('text'),  # Indicamos donde estará el texto dentro del dataframe
    get_y=ColReader('label'), # Indicamos cómo extraer la clase del dataframe
    splitter=ColSplitter('set') # Partimos el dataset en entrenamiento y validación
)

In [ ]:
dls = sentiment_clas.dataloaders(train_valid_df, bs=64)

In [ ]:
dls.show_batch(max_n=2)

In [ ]:
callbacks = [ShowGraphCallback(),
             SaveModelCallback()]

learnClass = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy,cbs=callbacks).to_fp16()

In [ ]:
learnClass = learnClass.load_encoder('finetuned')

In [ ]:
learnClass.fine_tune(10, 6e-2)

In [ ]:
learnClass.predict('with pale blue berries. in these peaceful shades--.')

In [ ]:
test_df['set']=True
test_df = test_df.drop(columns=['id'],axis=1)
train_test_df = pd.concat([train_df,test_df])
train_test_df = train_test_df.rename(columns={"verse_text": "text"})
dls_test = sentiment_clas.dataloaders(train_test_df, bs=64)

In [ ]:
learnClass.dls = dls_test
learnClass.validate()